Importing dependencies

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\balam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data preprocessing


In [4]:
dset=pd.read_csv('FA-KES-Dataset.csv',encoding="latin1")

In [5]:
dset.sample(5)

,unit_id,article_title,article_content,source,date,location,labels
295,1924058051,Barrel bombs kill 11 civilians in Daesh town i...,Last updated at Jul 172015 Barrel bombs droppe...,jordantimes,7/19/2015,aleppo,0
787,1965511206,Syrian Army Retakes Qaryatain City in Homs Kil...,03-04-2016 Syrian Army Retakes Qaryatain City ...,manar,4/3/2016,hama,0
373,1924058173,Coalition airstrikes kill 85 civilians in Daes...,Published July 19 2016 Airstrikes on Daesh-hel...,dailysabah,7/6/2016,aleppo,0
269,1923848303,Bomb in Syrias Idlib kills 30 government fight...,Bomb in Syrias Idlib kills 30 government fight...,ahram,5/6/2014,idlib,0
803,1965511231,Syrian Army Kills 48 ISIL Terrorists in Deir E...,April 6 2017 Syrian Army Kills 48 ISIL Terrori...,manar,4/4/2017,deir ezzor,1


In [6]:
dset.shape

(804, 7)

In [7]:
dset['labels'].value_counts()

labels
1    426
0    378
Name: count, dtype: int64

In [8]:
dset.isnull().sum()


unit_id            0
article_title      0
article_content    0
source             0
date               0
location           0
labels             0
dtype: int64

merging author and title

In [9]:
dset['content']=dset['article_title']+' '+dset['source']

In [11]:
dset.head()


,unit_id,article_title,article_content,source,date,location,labels,content
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0,Syria attack symptoms consistent with nerve ag...
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0,Homs governor says U.S. attack caused deaths b...
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0,Death toll from Aleppo bomb attack at least 11...
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0,Aleppo bomb blast kills six Syrian state TV nna
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo,0,29 Syria Rebels Dead in Fighting for Key Alepp...


In [12]:
X=dset.drop(columns='labels',axis=1)
Y=dset['labels']

In [13]:
print(Y)

0      0
1      0
2      0
3      0
4      0
      ..
799    1
800    1
801    0
802    1
803    1
Name: labels, Length: 804, dtype: int64


In [14]:
port_stem=PorterStemmer()

In [30]:
def stem(content):
     #removing non strings using re.sub
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    return " ".join(stemmed_content)

In [33]:
def stem1(content):
    try:
        # if content is a list → join into string
        if isinstance(content, list):
            content = " ".join(content)

        # if content is missing/NaN → return empty string
        if not isinstance(content, str):
            return ""

        # clean + preprocess
        stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
        stemmed_content = stemmed_content.lower()
        stemmed_content = stemmed_content.split()
        stemmed_content = [
            port_stem.stem(word)
            for word in stemmed_content
            if word not in stopwords.words('english')
        ]
        return " ".join(stemmed_content)

    except Exception as e:
        print(f"Error while processing: {content} → {e}")
        return ""


In [34]:
con="yria attack symptoms consistent wit333-3h3dc ewf203fpcpo-ek3p-0 232n312errfvv3fbk[okgk233e ag.."
stem1(con)

'yria attack symptom consist wit h dc ewf fpcpo ek p n errfvv fbk okgk e ag'

In [35]:
dset['content']=dset['content'].apply(stem1)

In [36]:
X=dset['content'].values
Y=dset['labels'].values

In [37]:
print(X)

['syria attack symptom consist nerv agent use nna'
 'hom governor say u attack cau death doesnt see big human loss nna'
 'death toll aleppo bomb attack least nna'
 'aleppo bomb blast kill six syrian state tv nna'
 'syria rebel dead fight key aleppo road nna'
 'suicid bomb kill least northeast syria nna'
 'dead heavi u raid syria stronghold nna'
 'suicid bomber kill assad clan hometown nna'
 'explo rock town damascu nna' 'damascu explo due rocket bomb nna'
 'syrian regim step aerial assault douma alarabi'
 'hizballah lead regim offen southern syria alarabi'
 'syrian opposit remain divid alarabi'
 'video show murder syrian activist alarabi'
 'syria nusra front stage deadli suicid bomb aleppo alarabi'
 'regim troop thwart rebel attack syria aleppo alarabi'
 'ahrar al sham leader kill syria alarabi'
 'barrel bomb kill town syria alarabi'
 'rebel advanc north western syria alarabi'
 'isra strike syrian town kill pro regim fighter alarabi'
 'syria armi plane crash rebel held town alarabi'
 '

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
 from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [41]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7095 stored elements and shape (804, 771)>
  Coords	Values
  (0, 21)	0.4398385037739128
  (0, 71)	0.154270763028172
  (0, 156)	0.4398385037739128
  (0, 463)	0.4143530883600743
  (0, 469)	0.31752877227058074
  (0, 671)	0.4398385037739128
  (0, 672)	0.1123974588039333
  (0, 734)	0.33268695746858934
  (1, 71)	0.12184316409612725
  (1, 95)	0.34738477945645097
  (1, 122)	0.3272563792892771
  (1, 188)	0.24114602484679362
  (1, 211)	0.34738477945645097
  (1, 299)	0.30189759411838357
  (1, 322)	0.20673629748416938
  (1, 332)	0.34738477945645097
  (1, 405)	0.30189759411838357
  (1, 469)	0.2507844619783276
  (1, 600)	0.22200068141769194
  (1, 609)	0.34738477945645097
  (2, 36)	0.21999407466746793
  (2, 71)	0.23968501507821843
  (2, 99)	0.2929163001914418
  (2, 188)	0.47437284668556384
  (2, 392)	0.3389939588680877
  :	:
  (800, 619)	0.34302181140418664
  (800, 689)	0.2062183140289721
  (801, 36)	0.24495050157124273
  (801, 71)	0.26687

In [50]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,stratify=Y,random_state=2)

In [51]:
X_train.shape
Y_train.shape

(723,)

In [52]:
model=LogisticRegression()
model.fit(X_train,Y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


ACCURACY

In [53]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)
print(training_data_accuracy)

0.7814661134163209


In [54]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)
print(test_data_accuracy)

0.5925925925925926


In [63]:
def predictive(content):
    processed = stem(content)
    transformed = vectorizer.transform([processed])
    prediction=model.predict(X_train[398])
    if(prediction[0]==0):
        print("The news is real")
    elif(prediction[0]==1):
        print("the news is not real")
predictive("ognngnpnrgpnprngnpj onoifnenfnew onpi2no  e pnp3o2 eoi2n3po3p2nip3eni2nerin3n")        

the news is not real


In [62]:
dset.sample(10)

,unit_id,article_title,article_content,source,date,location,labels,content
410,1926478915,18 Civilians Killed in Barrel Bomb Attack on R...,Wednesday 22 July 2015 18 Civilians Killed in ...,etilaf,7/22/2015,aleppo,1,civilian kill barrel bomb attack rural aleppo ...
384,1924058192,11 killed 25 injured in Russian air strike in ...,Published October 15 2016 At least 11 people w...,dailysabah,10/4/2016,idlib,1,kill injur russian air strike syria idlib dail...
287,1924058036,Outrage as Syria _chemical attack_ kills doz...,Last updated at Apr 052017 KHAN SHEIKHUN Syria...,jordantimes,4/15/2017,idlib,1,outrag syria chemic attack kill dozen rebel he...
502,1926479154,Assad Regime Commits Horrific Massacres After ...,Saturday 11 April 2015 18:05 Assad Regime Comm...,etilaf,2/2/2015,idlib,1,assad regim commit horrif massacr loss idlib e...
480,1926479096,18 Civilians Killed in Barrel Bomb Attack on R...,Wednesday 22 July 2015 22:33 18 Civilians Kill...,etilaf,7/22/2015,aleppo,0,civilian kill barrel bomb attack rural aleppo ...
106,1923102921,Syrian children killed in missile attack on sc...,Date of publication 16 September 2016 Syrian a...,alaraby,9/16/2016,deir ezzor,1,syrian children kill missil attack school alarabi
501,1926479154,Assad Regime Commits Horrific Massacres After ...,Saturday 11 April 2015 18:05 Assad Regime Comm...,etilaf,2/2/2015,idlib,0,assad regim commit horrif massacr loss idlib e...
398,1924058212,Regime shelling kills 10 civilians including 6...,Published November 17 2017 Shelling by the Ass...,dailysabah,11/6/2017,damascus,1,regim shell kill civilian includ children syri...
463,1926479049,6 citizens killed 22 injured in terrorist atta...,18 November 2016 6 citizens killed 22 injured ...,sana,11/18/2016,aleppo,0,citizen kill injur terrorist attack shell dama...
491,1926479115,Syrian Coalition Declares Idlib Disaster Area ...,Thursday 02 June 2016 16:02 Syrian Coalition D...,etilaf,6/2/2016,idlib,0,syrian coalit declar idlib disast area amid de...
